In [107]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'Gulim'

import warnings
warnings.filterwarnings(action='ignore')

In [108]:
from workalendar.asia import SouthKorea
import pendulum

In [109]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## 전처리
- 일자에서 월과 일을 분리
- 요일을 레이블 인코딩화(EDA로 요일의 중요도 순 파악)
- 월 별, 일 별 중식 석식 수요 차이 파악

In [110]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month
train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week
train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

train['출근'] = train['본사정원수']-(train['본사휴가자수']+train['본사출장자수']+train['현본사소속재택근무자수'])
train['휴가비율'] = train['본사휴가자수']/train['본사정원수']
train['출장비율'] = train['본사출장자수']/train['본사정원수']
train['야근비율'] = train['본사시간외근무명령서승인건수']/train['출근']
train['재택비율'] = train['현본사소속재택근무자수']/train['본사정원수']

test['출근'] = test['본사정원수']-(test['본사휴가자수']+test['본사출장자수']+test['현본사소속재택근무자수'])
test['휴가비율'] = test['본사휴가자수']/test['본사정원수']
test['출장비율'] = test['본사출장자수']/test['본사정원수']
test['야근비율'] = test['본사시간외근무명령서승인건수']/test['출근']
test['재택비율'] = test['현본사소속재택근무자수']/test['본사정원수']

train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

# train['중식참여율'] = train['중식계'] / train['식사가능자수']

In [111]:
month_rank4dinner = {
    1: 11,
    2: 2,
    3: 1,
    4: 4,
    5: 7,
    6: 6,
    7: 10,
    8: 8,
    9: 5,
    10: 3,
    11: 9,
    12: 12
}
train['월(석식)'] = train['월'].map(month_rank4dinner)
test['월(석식)'] = test['월'].map(month_rank4dinner)

month_rank4lunch = {
    1: 3,
    2: 1,
    3: 2,
    4: 6,
    5: 7,
    6: 8,
    7: 10,
    8: 9,
    9: 5,
    10: 4,
    11: 11,
    12: 12
}
train['월(중식)'] = train['월'].map(month_rank4lunch)
test['월(중식)'] = test['월'].map(month_rank4lunch)

weekday_rank4dinner = {
    '월': 1,
    '화': 2,
    '수': 4,
    '목': 3,
    '금': 5,
}

weekday_rank4lunch = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

train['요일(중식)'] = train['요일'].map(weekday_rank4lunch)
test['요일(중식)'] = test['요일'].map(weekday_rank4lunch)

In [112]:
rank = pd.DataFrame(range(1,53))
week_rank_lunch = pd.pivot_table(train,values='중식계',index='주').sort_values(by='중식계').reset_index().drop('중식계',axis=1)
week_rank_dinner = pd.pivot_table(train,values='석식계',index='주').sort_values(by='석식계').reset_index().drop('석식계',axis=1)


week_rank4lunch = {}
for i in range(len(rank)):
    week_rank4lunch[week_rank_lunch['주'][i]] = rank[0][i]


week_rank4dinner = {}
for i in range(len(rank)):
    week_rank4dinner[week_rank_dinner['주'][i]] = rank[0][i]
    
    
train['주(중식)'] = train['주'].map(week_rank4lunch)
test['주(중식)'] = test['주'].map(week_rank4lunch)

train['주(석식)'] = train['주'].map(week_rank4dinner)
test['주(석식)'] = test['주'].map(week_rank4dinner)

In [113]:
def is_holiday(date):
    holidays = list(map(str, pd.Series(np.array(SouthKorea().holidays(int(date[:4])))[:, 0])))
    
    yesterday = str(np.datetime64(date) - 1)
    tomorrow = str(np.datetime64(date) + 1)

    if tomorrow in holidays and yesterday in holidays:
        return 3
    if tomorrow in holidays:
        return 2
    elif yesterday in holidays:
        return 1
    else : 
        return 0

def week_of_month(x):
    dt = pendulum.parse(x)
    
    wom = dt.week_of_month
    if wom < 0:
        wom += 52
    return wom
    

df = pd.concat([train[['본사정원수', '일자']], test[['본사정원수', '일자']]])
df['년월'] = df['일자'].apply(lambda x : x[:7])
df = df[['년월', '본사정원수']].groupby(by=['년월'], as_index=False).mean()

def member_change(date):
    this_month = date[:7]
    last_month = str(np.datetime64(this_month) - 1)
    
    this_month_member = int(df[df['년월'] == this_month]['본사정원수'])
    last_month_member = int(df[df['년월'] == last_month]['본사정원수'])
    
    
    return  this_month_member - last_month_member

train['공휴일전후'] = train['일자'].apply(is_holiday)
test['공휴일전후'] = test['일자'].apply(is_holiday)

train['몇주차'] = train['일자'].apply(week_of_month)
test['몇주차'] = test['일자'].apply(week_of_month)

train = train[train['일자'] > '2016-03']
train['인원변화'] = train['일자'].apply(member_change)
test['인원변화'] = test['일자'].apply(member_change)

In [ ]:
menu_train = train[['조식메뉴','중식메뉴','석식메뉴']]
menu_test = test[['조식메뉴','중식메뉴','석식메뉴']]
print(menu_train.shape)
print(menu_test.shape)

In [ ]:
def clean_split(df):
    df = df.split()
    for j in df:
        if '(' in j:
            del df[df.index(j)]
    for i in df:
        if '쌀밥' in i:
            del df[df.index(i)] 
    for q in df:
        if '김치' in q:
            del df[df.index(q)]
    
    return df

In [ ]:
menu_train['중식메뉴_prepro'] = menu_train['중식메뉴'].apply(clean_split)
menu_test['중식메뉴_prepro'] = menu_test['중식메뉴'].apply(clean_split)

In [ ]:
lunch_train= menu_train.중식메뉴_prepro.tolist()
lunch_test= menu_test.중식메뉴_prepro.tolist()

In [ ]:
lunch_list_train = []
for i in lunch_train:
    if len(i) == 0:
        continue
    else:
        string = i[0]
    for w in i[1:]:
        string += " "
        string += w
    lunch_list_train.append(string)
lunch_list_train

In [ ]:
lunch_list_test = []
for i in lunch_test:
    if len(i) == 0:
        continue
    else:
        string = i[0]
    for w in i[1:]:
        string += " "
        string += w
    lunch_list_test.append(string)
lunch_list_test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
lunch_matrix_train = vectorizer.fit_transform(lunch_list_train)
lunch_df_train = pd.DataFrame(lunch_matrix_train.toarray(), columns=vectorizer.get_feature_names())

vectorizer = TfidfVectorizer()
lunch_matrix_test = vectorizer.fit_transform(lunch_list_test)
lunch_df_test = pd.DataFrame(lunch_matrix_test.toarray(), columns=vectorizer.get_feature_names())

In [ ]:
# 중복 메뉴 선정
lunch_intersection = list(set(lunch_df_train.columns) & set(lunch_df_test.columns))
print(len(lunch_intersection))

## 공휴일 변수 생성

In [114]:
train.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계', '월', '주', '일',
       '출근', '휴가비율', '출장비율', '야근비율', '재택비율', '식사가능자수', '월(석식)', '월(중식)',
       '요일(석식)', '요일(중식)', '주(중식)', '주(석식)', '공휴일전후', '몇주차', '인원변화'],
      dtype='object')

In [115]:
# 메뉴 변수 없이 사용할떄 해당 코드 사용['공휴일전후', '몇주차', '인원변화']

lunch_df_f = pd.concat([train[['공휴일전후', '몇주차', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','중식계']],lunch_df_train[lunch_intersection]],axis=1)
lunch_df_test_f = pd.concat([test[['공휴일전후', '몇주차', '요일(중식)','월(중식)','일','주(중식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수']],lunch_df_test[lunch_intersection]],axis=1)

lunch_train = lunch_df_f[['공휴일전후', '몇주차', '요일(중식)', '주(중식)', '출근', '출장비율', '야근비율', '재택비율',
                          '본사출장자수', '콩나물불고기', '동태매운탕', '파인d', '쇠고기숙주볶음', '삼색유자청무침', '소불고기', '바나나',
                          '황도d', '콩나물파채무침', '버섯매운탕', '돈육간장불고기', '양상추샐러드', '석박지', '수박', '오리대패불고기',
                          '오이생채', '흑임자d', '고등어구이', '닭볶음탕', '청경채찜', '우렁된장찌개', '요거트d','휴가비율']]
lunch_test = lunch_df_test_f



dinner_train= train[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]
dinner_test = test[['공휴일전후', '몇주차', '인원변화', '요일(석식)','월(석식)','일','주(석식)','출근', '휴가비율', '출장비율', '야근비율', '재택비율','본사출장자수','본사휴가자수','식사가능자수','본사시간외근무명령서승인건수']]

y_lunch = np.array(train[['중식계']])
y_dinner = np.array(train[['석식계']])

In [116]:
lunch_train.columns

Index(['공휴일전후', '몇주차', '인원변화', '요일(중식)', '월(중식)', '일', '주(중식)', '출근', '휴가비율',
       '출장비율', '야근비율', '재택비율', '본사출장자수', '본사휴가자수', '식사가능자수', '본사시간외근무명령서승인건수'],
      dtype='object')

In [117]:
print(lunch_train.shape)
print(lunch_test.shape)
print(dinner_train.shape)
print(dinner_test.shape)

(1187, 16)
(50, 16)


In [119]:
cat_features = [f for f in lunch_train.columns if lunch_train[f].dtype == 'object']

def column_index(df, cat_features):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, cat_features, sorter=sidx)]

cat_features_idx = column_index(lunch_train, cat_features)    
print("Cat features are: %s" % [f for f in cat_features])
print(cat_features_idx)

Cat features are: []
[]


#### 분포 확인 및 분포 조정

# 중식 예측모델

In [121]:
import optuna
from lightgbm import LGBMRegressor
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold

In [122]:
# from automl_alex import LightGBMRegressor, CatBoostRegressor, AutoMLRegressor
from sklearn.metrics import mean_absolute_error as MAE
import sklearn

In [125]:
def objective_lunch(trial: Trial) -> float:    
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "metric": "mae",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(lunch_train):
        X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
        y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

        model = LGBMRegressor(**params_lgb)

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  eval_metric="mae",
                  early_stopping_rounds=100,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold
    
    return mae

In [126]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_lunch, n_trials=1000)
lunch_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-22 20:44:09,235] A new study created in memory with name: lgbm_parameter_opt
[I 2021-07-22 20:44:10,090] Trial 0 finished with value: 84.95366638127439 and parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 15, 'num_leaves': 154, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'subsample_freq': 1, 'min_child_samples': 88, 'max_bin': 380}. Best is trial 0 with value: 84.95366638127439.
[I 2021-07-22 20:44:10,701] Trial 1 finished with value: 71.14812960335499 and parameters: {'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'num_leaves': 214, 'colsample_bytree': 0.5274034664069657, 'subsample': 0.42727747704497043, 'subsample_freq': 2, 'min_child_samples': 34, 'max_bin': 357}. Best is trial 1 with value: 71.14812960335499.
[I 2021-07-22 20:44:11,260] Trial 2 finished with value: 73.83578303036558 and parameters: {'reg_alpha': 1.2964031109077052e-05, 'reg_lambda': 0

[I 2021-07-22 20:44:21,943] Trial 21 finished with value: 67.2617211110972 and parameters: {'reg_alpha': 2.4762598256020862e-05, 'reg_lambda': 0.03896718859081358, 'max_depth': 7, 'num_leaves': 3, 'colsample_bytree': 0.4316672872051123, 'subsample': 0.8651575094611166, 'subsample_freq': 3, 'min_child_samples': 5, 'max_bin': 479}. Best is trial 14 with value: 66.26471601459959.
[I 2021-07-22 20:44:22,586] Trial 22 finished with value: 69.03250342800179 and parameters: {'reg_alpha': 2.2117257930089717e-05, 'reg_lambda': 0.04425504059520203, 'max_depth': 7, 'num_leaves': 22, 'colsample_bytree': 0.44383255653029213, 'subsample': 0.7365083005625545, 'subsample_freq': 1, 'min_child_samples': 14, 'max_bin': 498}. Best is trial 14 with value: 66.26471601459959.
[I 2021-07-22 20:44:23,097] Trial 23 finished with value: 66.14995864230615 and parameters: {'reg_alpha': 2.9914282003814064e-05, 'reg_lambda': 0.03506338856020266, 'max_depth': 4, 'num_leaves': 59, 'colsample_bytree': 0.549965426346235

[I 2021-07-22 20:44:35,011] Trial 42 finished with value: 71.85798900729746 and parameters: {'reg_alpha': 1.6949243313989294e-05, 'reg_lambda': 0.06379642809033942, 'max_depth': 1, 'num_leaves': 122, 'colsample_bytree': 0.8440245997299023, 'subsample': 0.6474507429109624, 'subsample_freq': 6, 'min_child_samples': 12, 'max_bin': 444}. Best is trial 41 with value: 64.87914909237617.
[I 2021-07-22 20:44:35,502] Trial 43 finished with value: 66.71202135173745 and parameters: {'reg_alpha': 8.966976787690099e-06, 'reg_lambda': 0.05594001378706654, 'max_depth': 4, 'num_leaves': 76, 'colsample_bytree': 0.7875958887611415, 'subsample': 0.5314694094884045, 'subsample_freq': 5, 'min_child_samples': 5, 'max_bin': 424}. Best is trial 41 with value: 64.87914909237617.
[I 2021-07-22 20:44:36,023] Trial 44 finished with value: 66.52220441787266 and parameters: {'reg_alpha': 1.5875484284466563e-05, 'reg_lambda': 0.07452006921638579, 'max_depth': 3, 'num_leaves': 56, 'colsample_bytree': 0.81840178821007

[I 2021-07-22 20:44:46,231] Trial 63 finished with value: 69.1144996182527 and parameters: {'reg_alpha': 5.09402086064609e-07, 'reg_lambda': 0.07681952128514323, 'max_depth': 3, 'num_leaves': 117, 'colsample_bytree': 0.8033576276342398, 'subsample': 0.6767179196076936, 'subsample_freq': 7, 'min_child_samples': 21, 'max_bin': 236}. Best is trial 41 with value: 64.87914909237617.
[I 2021-07-22 20:44:46,706] Trial 64 finished with value: 65.54524962831765 and parameters: {'reg_alpha': 6.317137660247566e-06, 'reg_lambda': 0.08768286680678533, 'max_depth': 4, 'num_leaves': 81, 'colsample_bytree': 0.8647766540051874, 'subsample': 0.6138828027016929, 'subsample_freq': 5, 'min_child_samples': 13, 'max_bin': 209}. Best is trial 41 with value: 64.87914909237617.
[I 2021-07-22 20:44:47,186] Trial 65 finished with value: 66.46619041707363 and parameters: {'reg_alpha': 1.641042746729113e-05, 'reg_lambda': 0.08824938733659593, 'max_depth': 4, 'num_leaves': 47, 'colsample_bytree': 0.9064811588992978,

[I 2021-07-22 20:44:57,603] Trial 84 finished with value: 65.9665574286277 and parameters: {'reg_alpha': 7.686088619667581e-08, 'reg_lambda': 0.0789595608786588, 'max_depth': 3, 'num_leaves': 50, 'colsample_bytree': 0.7769111099874437, 'subsample': 0.7224732591063583, 'subsample_freq': 5, 'min_child_samples': 10, 'max_bin': 302}. Best is trial 81 with value: 64.72862849893305.
[I 2021-07-22 20:44:58,057] Trial 85 finished with value: 76.623236767048 and parameters: {'reg_alpha': 3.07523877419542e-06, 'reg_lambda': 0.08761398590971205, 'max_depth': 1, 'num_leaves': 42, 'colsample_bytree': 0.7510503957484309, 'subsample': 0.6887672162350872, 'subsample_freq': 5, 'min_child_samples': 67, 'max_bin': 283}. Best is trial 81 with value: 64.72862849893305.
[I 2021-07-22 20:44:58,670] Trial 86 finished with value: 67.20931416664934 and parameters: {'reg_alpha': 1.7292134302253017e-05, 'reg_lambda': 0.08462695077973165, 'max_depth': 5, 'num_leaves': 93, 'colsample_bytree': 0.7241967770134575, 's

...

[I 2021-07-22 20:52:29,321] Trial 924 finished with value: 65.64523675039678 and parameters: {'reg_alpha': 1.2067980373252934e-05, 'reg_lambda': 0.00040297368544515294, 'max_depth': 3, 'num_leaves': 20, 'colsample_bytree': 0.874584774378881, 'subsample': 0.6850156232110611, 'subsample_freq': 5, 'min_child_samples': 12, 'max_bin': 452}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:29,855] Trial 925 finished with value: 65.530593351261 and parameters: {'reg_alpha': 1.475024752920682e-05, 'reg_lambda': 0.06843810912666375, 'max_depth': 2, 'num_leaves': 35, 'colsample_bytree': 0.85854284293083, 'subsample': 0.6603374254236664, 'subsample_freq': 5, 'min_child_samples': 7, 'max_bin': 442}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:30,371] Trial 926 finished with value: 66.25801740230868 and parameters: {'reg_alpha': 1.6247626611177668e-05, 'reg_lambda': 0.07250846955127992, 'max_depth': 4, 'num_leaves': 7, 'colsample_bytree': 0.8417871099705

[I 2021-07-22 20:52:40,474] Trial 945 finished with value: 65.72866540037492 and parameters: {'reg_alpha': 1.6479919954342643e-05, 'reg_lambda': 0.0642518884138307, 'max_depth': 4, 'num_leaves': 35, 'colsample_bytree': 0.8282387266154813, 'subsample': 0.6445967838135566, 'subsample_freq': 5, 'min_child_samples': 5, 'max_bin': 449}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:40,956] Trial 946 finished with value: 66.07603968995708 and parameters: {'reg_alpha': 1.2977227427308475e-05, 'reg_lambda': 0.06705428593553801, 'max_depth': 3, 'num_leaves': 21, 'colsample_bytree': 0.8570282032214207, 'subsample': 0.6244805573495484, 'subsample_freq': 6, 'min_child_samples': 9, 'max_bin': 439}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:41,380] Trial 947 finished with value: 65.21673458468258 and parameters: {'reg_alpha': 1.5835006926579525e-05, 'reg_lambda': 0.07694015300927833, 'max_depth': 3, 'num_leaves': 183, 'colsample_bytree': 0.8785975579

[I 2021-07-22 20:52:52,386] Trial 966 finished with value: 66.65617103767784 and parameters: {'reg_alpha': 1.5319545601573927e-05, 'reg_lambda': 0.07811372103002148, 'max_depth': 3, 'num_leaves': 141, 'colsample_bytree': 0.6656549996285395, 'subsample': 0.6283949397460575, 'subsample_freq': 4, 'min_child_samples': 8, 'max_bin': 462}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:52,931] Trial 967 finished with value: 67.56640808930803 and parameters: {'reg_alpha': 1.3201160966394747e-05, 'reg_lambda': 0.07228912778960139, 'max_depth': 4, 'num_leaves': 29, 'colsample_bytree': 0.8818525559281782, 'subsample': 0.6443120865602244, 'subsample_freq': 6, 'min_child_samples': 13, 'max_bin': 470}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:52:53,366] Trial 968 finished with value: 64.63723310159494 and parameters: {'reg_alpha': 1.44670099626179e-05, 'reg_lambda': 0.06752414461847642, 'max_depth': 3, 'num_leaves': 63, 'colsample_bytree': 0.8681785155

[I 2021-07-22 20:53:04,439] Trial 987 finished with value: 71.6079343517857 and parameters: {'reg_alpha': 1.4848634237784156e-05, 'reg_lambda': 0.07383132185081812, 'max_depth': 1, 'num_leaves': 27, 'colsample_bytree': 0.8661899437178494, 'subsample': 0.636924133761817, 'subsample_freq': 3, 'min_child_samples': 5, 'max_bin': 492}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:53:05,001] Trial 988 finished with value: 66.1297591417528 and parameters: {'reg_alpha': 1.4008431247987877e-05, 'reg_lambda': 0.0699736757226336, 'max_depth': 2, 'num_leaves': 32, 'colsample_bytree': 0.856661701082953, 'subsample': 0.6654706386296925, 'subsample_freq': 4, 'min_child_samples': 7, 'max_bin': 412}. Best is trial 200 with value: 63.67949352328331.
[I 2021-07-22 20:53:05,584] Trial 989 finished with value: 65.65193556602503 and parameters: {'reg_alpha': 1.5781548727324067e-05, 'reg_lambda': 0.07423053471177664, 'max_depth': 2, 'num_leaves': 39, 'colsample_bytree': 0.894549962546961

Best Score: 63.67949352328331
Best trial: {'reg_alpha': 1.4915710375825325e-05, 'reg_lambda': 0.068960929940221, 'max_depth': 3, 'num_leaves': 59, 'colsample_bytree': 0.8507489233182559, 'subsample': 0.6530080107795074, 'subsample_freq': 5, 'min_child_samples': 5, 'max_bin': 436}


In [133]:
lunch_mae = 0
lunch_pred = np.array([0]*50).astype('float64')
n_fold = 12
kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)

for train_index, test_index in kf.split(lunch_train):
    X_train, X_valid = lunch_train.iloc[train_index], lunch_train.iloc[test_index]
    y_train, y_valid = y_lunch[train_index], y_lunch[test_index]

    model = LGBMRegressor(**lunch_param)

    evaluation = [(X_train, y_train), (X_valid, y_valid)]

    model.fit(X_train, y_train,
              eval_set=evaluation, 
              eval_metric="mae",
              early_stopping_rounds=100,
              verbose=0)

    pred = model.predict(X_valid)
    lunch_mae += MAE(y_valid, pred)/n_fold
    
    lunch_pred += model.predict(lunch_test)/n_fold
    
print(lunch_mae)

66.61667942718009


In [130]:
def objective_dinner(trial: Trial) -> float: 
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "metric": "mae",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    n_fold = 4
    kf = KFold(n_splits=n_fold, random_state=42, shuffle=True)
    mae_log = 0
    
    for train_index, test_index in kf.split(dinner_train):
        X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
        y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

        model = LGBMRegressor(**params_lgb)

        evaluation = [(X_train, y_train), (X_valid, y_valid)]

        model.fit(X_train, y_train,
                  eval_set=evaluation, 
                  eval_metric="mae",
                  early_stopping_rounds=100,
                  verbose=0)

        pred = model.predict(X_valid)
        mae= MAE(y_valid, pred)   
        
        mae_log += mae/n_fold
    
    return mae_log

In [131]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective_dinner, n_trials=1000)
dinner_param = study.best_trial.params

print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2021-07-22 20:54:13,691] A new study created in memory with name: lgbm_parameter_opt
[I 2021-07-22 20:54:14,155] Trial 0 finished with value: 69.81429415341069 and parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 15, 'num_leaves': 154, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'subsample_freq': 1, 'min_child_samples': 88, 'max_bin': 380}. Best is trial 0 with value: 69.81429415341069.
[I 2021-07-22 20:54:14,658] Trial 1 finished with value: 59.892280502620764 and parameters: {'reg_alpha': 2.1245096608103405e-05, 'reg_lambda': 0.0018526142807772773, 'max_depth': 20, 'num_leaves': 214, 'colsample_bytree': 0.5274034664069657, 'subsample': 0.42727747704497043, 'subsample_freq': 2, 'min_child_samples': 34, 'max_bin': 357}. Best is trial 1 with value: 59.892280502620764.
[I 2021-07-22 20:54:15,197] Trial 2 finished with value: 62.184481451901746 and parameters: {'reg_alpha': 1.2964031109077052e-05, 'reg_lambda'

[I 2021-07-22 20:54:29,526] Trial 21 finished with value: 54.32076750437723 and parameters: {'reg_alpha': 4.70434463781003e-06, 'reg_lambda': 0.07350330976786656, 'max_depth': 5, 'num_leaves': 106, 'colsample_bytree': 0.947272124260215, 'subsample': 0.766394870673477, 'subsample_freq': 5, 'min_child_samples': 5, 'max_bin': 344}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:30,002] Trial 22 finished with value: 56.00475065566729 and parameters: {'reg_alpha': 3.631430534631998e-06, 'reg_lambda': 0.04360021869054573, 'max_depth': 5, 'num_leaves': 110, 'colsample_bytree': 0.8290554365627579, 'subsample': 0.6285305113225365, 'subsample_freq': 5, 'min_child_samples': 25, 'max_bin': 335}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:30,579] Trial 23 finished with value: 55.57916616075577 and parameters: {'reg_alpha': 4.49797593854285e-06, 'reg_lambda': 0.07525067836888842, 'max_depth': 9, 'num_leaves': 89, 'colsample_bytree': 0.9367232822483036, '

[I 2021-07-22 20:54:41,680] Trial 42 finished with value: 54.776139076457895 and parameters: {'reg_alpha': 1.1252601224900237e-05, 'reg_lambda': 0.06457503739832876, 'max_depth': 4, 'num_leaves': 130, 'colsample_bytree': 0.8714609900946467, 'subsample': 0.8974732409633823, 'subsample_freq': 5, 'min_child_samples': 19, 'max_bin': 352}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:42,201] Trial 43 finished with value: 55.42463308676308 and parameters: {'reg_alpha': 1.0964327312817006e-05, 'reg_lambda': 0.06387726942732377, 'max_depth': 3, 'num_leaves': 117, 'colsample_bytree': 0.8803892905312154, 'subsample': 0.8929537062010169, 'subsample_freq': 4, 'min_child_samples': 24, 'max_bin': 346}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:42,742] Trial 44 finished with value: 55.37186225810416 and parameters: {'reg_alpha': 8.923403712991337e-06, 'reg_lambda': 0.07790812571269827, 'max_depth': 4, 'num_leaves': 133, 'colsample_bytree': 0.9140089720

[I 2021-07-22 20:54:53,409] Trial 63 finished with value: 54.983686015940464 and parameters: {'reg_alpha': 1.213534619880429e-05, 'reg_lambda': 0.058479552003110405, 'max_depth': 4, 'num_leaves': 113, 'colsample_bytree': 0.8951619453535937, 'subsample': 0.9030964536455186, 'subsample_freq': 4, 'min_child_samples': 12, 'max_bin': 341}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:53,978] Trial 64 finished with value: 55.12260365969014 and parameters: {'reg_alpha': 1.8993053290283278e-05, 'reg_lambda': 0.08187705896580064, 'max_depth': 8, 'num_leaves': 128, 'colsample_bytree': 0.8633299262508464, 'subsample': 0.9940952973877772, 'subsample_freq': 6, 'min_child_samples': 21, 'max_bin': 394}. Best is trial 21 with value: 54.32076750437723.
[I 2021-07-22 20:54:54,431] Trial 65 finished with value: 55.81465565488213 and parameters: {'reg_alpha': 1.5016224627909785e-06, 'reg_lambda': 0.07759543508186448, 'max_depth': 3, 'num_leaves': 85, 'colsample_bytree': 0.7975282725

[I 2021-07-22 20:55:04,306] Trial 84 finished with value: 54.39441134711661 and parameters: {'reg_alpha': 9.004483222098297e-06, 'reg_lambda': 0.07993248527401273, 'max_depth': 5, 'num_leaves': 49, 'colsample_bytree': 0.6400099362529585, 'subsample': 0.7336701501357319, 'subsample_freq': 8, 'min_child_samples': 8, 'max_bin': 227}. Best is trial 81 with value: 54.24945885925817.
[I 2021-07-22 20:55:04,785] Trial 85 finished with value: 55.06492152498987 and parameters: {'reg_alpha': 8.684875272847854e-06, 'reg_lambda': 0.0802666467413452, 'max_depth': 5, 'num_leaves': 31, 'colsample_bytree': 0.6503404853254491, 'subsample': 0.656094248075856, 'subsample_freq': 8, 'min_child_samples': 9, 'max_bin': 224}. Best is trial 81 with value: 54.24945885925817.
[I 2021-07-22 20:55:05,340] Trial 86 finished with value: 56.15880700129533 and parameters: {'reg_alpha': 9.44741815876297e-06, 'reg_lambda': 0.08409770982214107, 'max_depth': 3, 'num_leaves': 45, 'colsample_bytree': 0.6377173797597051, 'su

...

[I 2021-07-22 21:02:54,872] Trial 924 finished with value: 59.54250994391367 and parameters: {'reg_alpha': 2.1223554480340565e-05, 'reg_lambda': 0.07832976536970132, 'max_depth': 2, 'num_leaves': 36, 'colsample_bytree': 0.8014032243941451, 'subsample': 0.8199097758480479, 'subsample_freq': 5, 'min_child_samples': 8, 'max_bin': 491}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:02:55,437] Trial 925 finished with value: 54.28592271068827 and parameters: {'reg_alpha': 1.560097207430514e-05, 'reg_lambda': 0.08196349566710392, 'max_depth': 5, 'num_leaves': 27, 'colsample_bytree': 0.7810792128394453, 'subsample': 0.7820664913867931, 'subsample_freq': 4, 'min_child_samples': 5, 'max_bin': 478}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:02:55,905] Trial 926 finished with value: 54.01525902522135 and parameters: {'reg_alpha': 2.042933428094943e-05, 'reg_lambda': 0.07618195418746451, 'max_depth': 4, 'num_leaves': 52, 'colsample_bytree': 0.7568987941

[I 2021-07-22 21:03:06,854] Trial 945 finished with value: 54.28869239201532 and parameters: {'reg_alpha': 1.7297646235133664e-05, 'reg_lambda': 0.07042159962678396, 'max_depth': 5, 'num_leaves': 38, 'colsample_bytree': 0.6962334154359988, 'subsample': 0.7239644680637338, 'subsample_freq': 5, 'min_child_samples': 5, 'max_bin': 477}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:07,480] Trial 946 finished with value: 59.56373855950123 and parameters: {'reg_alpha': 1.5794803917131613e-05, 'reg_lambda': 0.07445119944568038, 'max_depth': 5, 'num_leaves': 34, 'colsample_bytree': 0.7555607856786354, 'subsample': 0.8013075819645613, 'subsample_freq': 5, 'min_child_samples': 72, 'max_bin': 261}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:07,963] Trial 947 finished with value: 54.2788933935274 and parameters: {'reg_alpha': 1.3535111333260724e-05, 'reg_lambda': 0.05473510163138702, 'max_depth': 4, 'num_leaves': 48, 'colsample_bytree': 0.85011485

[I 2021-07-22 21:03:18,969] Trial 966 finished with value: 54.596415588014736 and parameters: {'reg_alpha': 2.1490898453390756e-05, 'reg_lambda': 0.06606327450026661, 'max_depth': 6, 'num_leaves': 68, 'colsample_bytree': 0.8180092520262544, 'subsample': 0.757916100320716, 'subsample_freq': 4, 'min_child_samples': 10, 'max_bin': 232}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:19,514] Trial 967 finished with value: 54.75735005001001 and parameters: {'reg_alpha': 1.9903079683245993e-05, 'reg_lambda': 0.07980128795367614, 'max_depth': 4, 'num_leaves': 53, 'colsample_bytree': 0.8622754141216306, 'subsample': 0.8113425606425764, 'subsample_freq': 5, 'min_child_samples': 7, 'max_bin': 489}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:20,101] Trial 968 finished with value: 54.86827599381975 and parameters: {'reg_alpha': 1.720192275749049e-05, 'reg_lambda': 0.08242199773584255, 'max_depth': 5, 'num_leaves': 59, 'colsample_bytree': 0.71350045

[I 2021-07-22 21:03:30,919] Trial 987 finished with value: 54.41008965116221 and parameters: {'reg_alpha': 1.4883395809299166e-05, 'reg_lambda': 0.060723741860131644, 'max_depth': 5, 'num_leaves': 29, 'colsample_bytree': 0.8011707258143551, 'subsample': 0.6926802013996229, 'subsample_freq': 5, 'min_child_samples': 7, 'max_bin': 253}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:31,612] Trial 988 finished with value: 54.98074118741354 and parameters: {'reg_alpha': 1.6996946056425122e-05, 'reg_lambda': 0.04813114143896759, 'max_depth': 7, 'num_leaves': 44, 'colsample_bytree': 0.7582422219554736, 'subsample': 0.7573284514790115, 'subsample_freq': 5, 'min_child_samples': 15, 'max_bin': 247}. Best is trial 747 with value: 53.270360935966885.
[I 2021-07-22 21:03:32,048] Trial 989 finished with value: 56.06283165377408 and parameters: {'reg_alpha': 2.1367424124115826e-05, 'reg_lambda': 0.05609571564467899, 'max_depth': 3, 'num_leaves': 40, 'colsample_bytree': 0.778913

Best Score: 53.270360935966885
Best trial: {'reg_alpha': 2.104265305702583e-05, 'reg_lambda': 0.053655695641087825, 'max_depth': 5, 'num_leaves': 77, 'colsample_bytree': 0.7880830675091722, 'subsample': 0.7299276538738746, 'subsample_freq': 7, 'min_child_samples': 7, 'max_bin': 500}


In [134]:
dinner_mae = 0
dinner_pred = np.array([0]*50).astype('float64')

for train_index, test_index in kf.split(dinner_train):
    X_train, X_valid = dinner_train.iloc[train_index], dinner_train.iloc[test_index]
    y_train, y_valid = y_dinner[train_index], y_dinner[test_index]

    model = LGBMRegressor(**dinner_param)

    evaluation = [(X_train, y_train), (X_valid, y_valid)]

    model.fit(X_train, y_train,
              eval_set=evaluation, 
              eval_metric="mae",
              early_stopping_rounds=100,
              verbose=0)

    pred = model.predict(X_valid)
    dinner_mae += MAE(y_valid, pred)/n_fold
    
    dinner_pred += model.predict(dinner_test)/n_fold
    
print(dinner_mae)

55.291175499910715


In [140]:
submission = pd.read_csv('../data/sample_submission.csv')

submission.iloc[:,1] = lunch_pred
submission.iloc[:,2] = dinner_pred
submission.head()

,일자,중식계,석식계
0,2021-01-27,1001.319987,308.383648
1,2021-01-28,917.049772,416.512028
2,2021-01-29,631.140523,216.336773
3,2021-02-01,1307.361551,529.224504
4,2021-02-02,1076.744619,454.459879


In [141]:
def compare_ans(DIR):
    answer = pd.read_csv(DIR)

    lunch_answer = np.array(answer.iloc[:,1])
    dinner_answer = np.array(answer.iloc[:,2])
    
    lunch_MAE = abs(submission.iloc[:,1] - lunch_answer).mean()
    dinner_MAE = abs(submission.iloc[:,2] - dinner_answer).mean()
    
    print("lunch_MAE : ", lunch_MAE)
    print("dinner_MAE : ", dinner_MAE)
    print("total_MAE : ", (lunch_MAE+dinner_MAE)/2)

In [142]:
import glob

files = glob.glob('../submission/*')

for file in files[:-1]:
    print(file, compare_ans(file), end="\n\n" )

lunch_MAE :  15.820819587251727
dinner_MAE :  25.241357404783212
total_MAE :  20.531088496017468
../submission\20210624_pycaret_(4-2)_제출해봐야할거.csv None

lunch_MAE :  47.07681193424787
dinner_MAE :  38.13295791472805
total_MAE :  42.60488492448796
../submission\20210625_cat.csv None

lunch_MAE :  40.029612085753186
dinner_MAE :  32.8677824322947
total_MAE :  36.448697259023945
../submission\20210625_cat_10fold.csv None

lunch_MAE :  38.56439927855176
dinner_MAE :  33.27237886581164
total_MAE :  35.9183890721817
../submission\20210627_cat_15_KJH.csv None

lunch_MAE :  59.30245160826436
dinner_MAE :  53.537829583883976
total_MAE :  56.42014059607417
../submission\20210627_cat_after_2020.csv None

lunch_MAE :  38.56439927855176
dinner_MAE :  33.27237886581164
total_MAE :  35.9183890721817
../submission\20210627_cat_KJH.csv None

lunch_MAE :  38.56439927855176
dinner_MAE :  33.27237886581164
total_MAE :  35.9183890721817
../submission\20210627_pycaret_KJH.csv None

lunch_MAE :  27.5537131598

# 저장

In [143]:
import datetime
today = str(datetime.datetime.now().date()).replace("-","")
print("오늘 날짜 : " + today)

submission.to_csv(f'../submission/{today}_optuna_lgbm.csv', index =False)

오늘 날짜 : 20210722


In [146]:
hyperOpt = pd.read_csv('../submission/20210722_hyperOpt_lgbm.csv')

In [150]:
submission_mix = submission.copy()

In [155]:
submission_mix.iloc[:,1] = hyperOpt.iloc[:,1]

In [160]:
submission_mix.head(4)

,일자,중식계,석식계
0,2021-01-27,1006.208305,308.383648
1,2021-01-28,924.785410,416.512028
2,2021-01-29,634.318068,216.336773
3,2021-02-01,1293.689435,529.224504


In [161]:
submission_mix.to_csv(f'../submission/{today}_mix_lgbm.csv', index =False)